In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# 벡터DB : Chroma vs. Pinecone
- Chroma : 인메모리 vector DB, 로컬 vector DB
- Pinecone : 클라우드 vector DB
    (https://www.pinecone.io에서 api key 생성 -> .env에 추가(PINECONE_API_KEY등록)

# 0. 패키지 설치

In [2]:
%pip install -q pinecone langchain-pinecone --no-warn-script-location

Note: you may need to restart the kernel to use updated packages.


# 1. knowledge Base 구성을 위한 데이터 생성

In [3]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader('data/소득세법(법률)(제21065호)(20260102).docx')
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, 
    chunk_overlap=200,
    # separators=["\n\n", "\n", " ", ""]
)
document_list = loader.load_and_split(text_splitter=text_splitter)
len(document_list)

193

In [4]:
# embedding : OpenAI API text-embedding-3-large
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
load_dotenv()
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [5]:
%%time
# pinecone vector database
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
import os
pc = Pinecone(
    api_key=os.getenv("PINECONE_API_KEY")
)
# 데이터를 처음 업로드할 때
index_name = "tax-index"
# database = PineconeVectorStore.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     index_name=index_name
# )
# 업로드한 벡터db를 가져올 때
database = PineconeVectorStore(
    embedding=embedding,# 질문을 임베딩하여 유사도 검색
    index_name=index_name
)

C:\Users\Admin\anaconda3\envs\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CPU times: total: 844 ms
Wall time: 2.35 s


In [6]:
import os
from pinecone import Pinecone

pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
print(pc.list_indexes())


[{
    "name": "tax-index",
    "metric": "cosine",
    "host": "tax-index-znww7g1.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": null
}]


In [7]:
index = pc.Index(index_name)
print(pc.describe_index(index_name))


{'deletion_protection': 'disabled',
 'dimension': 1536,
 'host': 'tax-index-znww7g1.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'tax-index',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'},
 'tags': None,
 'vector_type': 'dense'}


In [8]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # 1536


# 2. 답변 생성을 위한 Retrieval

In [9]:
query = "연봉이 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_docs = database.similarity_search(query, k=3) # 기본k값:4

In [10]:
# retrieved_docs[2].page_content
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

In [11]:
retriever = database.as_retriever(
    search_kwargs={"k":3}
)
retrived_docs = retriever.invoke(query)
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

# 3. 답변 생성

In [12]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = "gpt-4.1-nano")

In [13]:
# upstage에서 받은 20$로 llm을 사용하고 싶다면
from langchain_upstage import ChatUpstage
llm = ChatUpstage(
    model = "solar-pro2",
    reasoning_effort="high" #느리지만 더 깊게 추론함 (low, medium)
)

In [14]:
prompt = f"""[identity]
- 당신은 최고의 한국 소득세법 전문가입니다
- [context]를 참고해서 사용자의 질문에 답변해 주세요.
- [context]는 다음과 같아요
{retrieved_doc}
- 질문 : {query}"""

In [15]:
ai_message = llm.invoke(prompt)

In [16]:
print(ai_message.content)

연봉 5천만원인 직장인의 소득세를 계산할 때는 **과세표준**을 산정한 후 **누진세율**을 적용해야 합니다. 다음 단계에 따라 계산합니다:

---

### 1. **과세표준 산정**
   - **총소득**: 50,000,000원  
   - **필수 공제항목**:
     - **사회보험료**: 국민연금(4.5%) + 건강보험(3.545%) + 장기요양보험(0.459%) + 고용보험(0.8%) = **월급의 약 9.3%**
       - 연간 사회보험료: \( 50,000,000 \times 9.3\% \approx 4,650,000 \)원  
     - **기본공제**: 1,500,000원 (단독세대 기준)  
   - **과세표준**:  
     \[
     50,000,000 - 4,650,000 \,(\text{사회보험료}) - 1,500,000 \,(\text{기본공제}) = 43,850,000 \,원
     \]

---

### 2. **국세 계산 (누진세율 적용)**
   - **2023년 소득세 세율**:
     - 1,200만원 이하: 6%  
     - 1,200만원 초과 ~ 4,600만원 이하: 15%  
     - 4,600만원 초과 ~ 8,800만원 이하: 24%  
     - (상위 구간 생략)  
   - **계산**:  
     \[
     (12,000,000 \times 6\%) + (43,850,000 - 12,000,000) \times 15\% = 720,000 + 4,777,500 = 5,497,500 \,원 \,(\text{국세})
     \]

---

### 3. **지방세 계산**
   - 지방세는 국세의 **10%**입니다.  
     \[
     5,497,500 \times 10\% = 549,750 \,원
     \]

---

### 4. **총 소득세**
   \[
   5,497,500 \,(\text{국세}) + 549,750 \,(\text{지방세}) = \boxed